## A stateless oracle (1): getting the data
#### 09.1 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-15

* Part 1: get the data form Coingecko
* Parts 1-4 are only relevant if you want to **create** an Oracle
* Only parts 5-6 are needed to **use** the oracle.

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import time

## Getting the data: Coingecko API
- CoinGecko states they are "the world’s largest independent cryptocurrency data aggregator"
- https://www.coingecko.com/en/about
- Their API can be used without API key

In [ ]:
!pip install pycoingecko

### Connect to Coigecko

In [ ]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

## Obtain last price

In [ ]:
price = cg.get_price(ids='algorand', vs_currencies='usd')
print(price['algorand']['usd'])

## Appendix: Obtain and plot historical data from Coingecko
* Daily data for the last *xxx* days
* Date is given as UNIX time
* Use pandas to convert

In [ ]:
df = pd.DataFrame(cg.get_coin_market_chart_by_id('algorand',vs_currency = 'usd', days = '100')['prices'])
df.columns = ['date', 'ALGO']
df.tail(3)
# The last row is the current price. If you only want historical data you have to drop the last row
# df = df[:-1]

In [ ]:
#conversion of unix time to datetime
df['date'] = pd.to_datetime(df['date'],unit='ms').dt.floor('d')
df.head()

In [ ]:
# Make a plot
plt = df.plot(x='date',y='ALGO')